In [1]:
from sentence_transformers import SentenceTransformer, util

/home/davor/moj-chatbot-search/env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from langchain.document_loaders import DirectoryLoader

In [3]:
loader = DirectoryLoader("data/documents/", glob="*.pdf")

In [4]:
documents = loader.load()
# DOWNLOADS TO /home/davor/nltk_data

In [5]:
from langchain.text_splitter import SentenceTransformersTokenTextSplitter

In [6]:
splitter = SentenceTransformersTokenTextSplitter(
    chunk_overlap=50,
    model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
)

In [7]:
split_documents = splitter.split_documents(documents)

In [8]:
from langchain.embeddings import HuggingFaceEmbeddings

model_name = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": False}
embeddings_model = HuggingFaceEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)

In [9]:
from langchain.vectorstores import Chroma

db = Chroma.from_documents(split_documents, embeddings_model)
retriever = db.as_retriever()

In [14]:
retriever.invoke(
    """kako da nađem sudski predmet?"""
)

[Document(page_content='uju u sudskim postupcima, a obuhvaća predmete općinskih, županijskih, upravnih, trgovačkih sudova, Visokog prekršajnog suda, Visokog trgovačkog suda, Visokog upravnog suda, Visokog kaznenog suda i Vrhovnog suda RH. Na Internet stranici http://e-predmet.pravosudje.hr/?q=pretraga biranjem nadležnog suda sa padajućeg izbornika i upisom točnog broja predmeta, možete vidjeti status vašeg predmeta. Ministarstvo pravosuđa i uprave kao najviše tijelo za obavljanje pravosudne', metadata={'source': 'data/documents/Popis pitanja za chatbot.docx.pdf'}),
 Document(page_content='hr/?q=pretraga biranjem nadležnog suda sa padajućeg izbornika i upisom točnog broja predmeta, možete vidjeti status vašeg predmeta. Ministarstvo pravosuđa i uprave kao najviše tijelo za obavljanje pravosudne uprave osigurava samo tehničke preduvjete za javnu i besplatnu uslugu ePredmet. Svoje predmete sa svim detaljima i sadržajem pismena možete vidjeti eKomunikacija https://usluge.pravosudje.hr/komun

In [17]:
embeddings = embeddings_model.embed_documents([split_document.page_content for split_document in split_documents])

In [21]:
len(embeddings), len(embeddings[0])

(133, 384)

In [2]:
model = SentenceTransformer(
    "./data/model_data/sentence-transformers_paraphrase-multilingual-MiniLM-L12-v2",  # "paraphrase-multilingual-MiniLM-L12-v2",
)

In [3]:
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)